In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [2]:
# SentenceBert 모델 세팅
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/931 [00:00<?, ?B/s]

[[-0.37510487 -0.7733841   0.59277123 ...  0.57923526  0.3268348
  -0.65089655]
 [-0.09361712 -0.18191531 -0.19230822 ... -0.03165831  0.30412522
  -0.2679356 ]]


# 데이터셋 로드

In [4]:
df = pd.read_csv('wellness_dataset_original.csv')

print(df.shape)
df.head()

(5231, 4)


,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


# 전처리

In [5]:
# 불필요한 컬럼 삭제
df = df.drop(columns=['Unnamed: 3'])
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [9]:
# 챗봇(응답) 컬럼이 null이 아닌 것들만 남기기
df = df[~df['챗봇'].isna()]

print(df.shape)
df.head()

(1034, 3)


,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


# 유저 대화내용 인코딩

In [10]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [11]:
# 문장 인코딩(벡터 임베딩)
model.encode(df.loc[0, '유저'])

array([-4.80606765e-01, -2.94869095e-01,  4.37900394e-01, -6.40137672e-01,
        3.28668877e-02, -3.42647761e-01, -5.47482111e-02,  1.73055660e-02,
       -4.08220947e-01, -5.06033957e-01, -1.68733254e-01, -3.98677200e-01,
       -1.24776796e-01, -9.71538872e-02, -1.65286750e-01,  5.72632765e-03,
        6.13689907e-02, -1.91311732e-01,  2.53917694e-01, -5.85019410e-01,
       -2.84426004e-01, -2.32035056e-01, -3.27080727e-01,  6.72981292e-02,
       -1.65786059e-05, -4.72336233e-01, -3.60021979e-01,  2.91880161e-01,
       -6.63861156e-01, -3.10574949e-01,  5.79525113e-01, -3.11722606e-01,
        1.47697050e-02, -2.12172374e-01,  2.22058147e-01, -1.73828587e-01,
       -3.78458440e-01, -4.20397699e-01, -2.38218755e-01,  6.38707653e-02,
       -1.15304485e-01, -2.44563714e-01, -5.00228345e-01,  1.68355361e-01,
       -6.58360362e-01, -8.91942322e-01, -6.26956940e-01, -3.21965367e-01,
       -7.05358446e-01,  3.71447414e-01, -5.45803607e-01,  7.76300877e-02,
        1.09864548e-01,  

In [13]:
%%time
# 유저들의 말들을 임베딩하고 그 결과를 embedding 컬럼에 저장
df['embedding'] = pd.Series([[]] * len(df)) # dummy
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

CPU times: user 1min 23s, sys: 2.3 s, total: 1min 25s
Wall time: 1min 24s


,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060676, -0.2948691, 0.4379004, -0.640137..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561576, -0.1450624, 0.29490346, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520005, -0.081268206, 1.0945569, 0.10579..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.46520713, 0.5285071, -0.507604..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277758, -0.37413916, 0.040532213, -0.862..."


In [14]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [17]:
# 사용자 입력 텍스트 임베딩하기
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [18]:
# 사용자 입력 텍스트와 미리 입력된 임베딩과 유사한 문장 찾기(cosine_similarity 이용)
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060676, -0.2948691, 0.4379004, -0.640137...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561576, -0.1450624, 0.29490346, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520005, -0.081268206, 1.0945569, 0.10579...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.46520713, 0.5285071, -0.507604...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277758, -0.37413916, 0.040532213, -0.862...",0.315118


In [20]:
answer = df.loc[df['distance'].idxmax()]

print('구분: ', answer['구분'])
print('유사한 질문: ', answer['유저'])
print('챗봇 답변: ', answer['챗봇'])
print('유사도: ', answer['distance'])

구분:  증상/편두통
유사한 질문:  요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇 답변:  으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
유사도:  0.8296287059783936
